In [6]:
import gensim.downloader as api
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec

In [7]:
wv = api.load('word2vec-google-news-300')
df_synonym = pd.read_csv('synonym.csv')
df_synonym.columns
df_synonym.head(5)

,question,answer,0,1,2,3
0,enormously,tremendously,appropriately,uniquely,tremendously,decidedly
1,provisions,stipulations,stipulations,interrelations,jurisdictions,interpretations
2,haphazardly,randomly,dangerously,densely,randomly,linearly
3,prominent,conspicuous,battered,ancient,mysterious,conspicuous
4,zenith,pinnacle,completion,pinnacle,outset,decline


In [8]:
# General Methods for Task 1

def compute_max(model, target, word1, word2, word3, word4, answer):
    # Determines the cosine similarity between two embeddings and retrieves the closest one to the target (question-word)
    words = [word1, word2, word3,word4]
    max_similarity = -1  
    selected_word = None

    for word in words:
        try:
            similarity = model.similarity(word, target)
            if similarity > max_similarity:
                # Keep the word with the greatest similarity to the target word
                max_similarity = similarity
                selected_word = word
        except KeyError:
            print(f"KeyError: {word}")
            continue
    if selected_word is None or target not in model.index_to_key or all(word not in model.index_to_key for word in words):
        label = 'guess'
    elif selected_word == answer:
        label = 'correct'
    else:
        label = 'wrong'

    return selected_word, label



def write_model_to_csv(name,model):
    # Writes a dataframe to a file demonstrating the best predicted synonym based on a question-word.
    # It also indicates whether the model correctly labeled the expected predicted word.

    df_synonym = pd.read_csv('synonym.csv')

    result_df = {
        'question' : [],
        'answer' : [],
        'predictions': [],
        'label': []
    }

    result_df['question'] = df_synonym['question']
    result_df['answer'] = df_synonym['answer']

    predictions = []
    labels = []
    
    for index in range(len(df_synonym)):
        prediction, label = compute_max(model, str(df_synonym['question'][index]), str(df_synonym['0'][index]),str(df_synonym['1'][index]), str(df_synonym['2'][index]),str(df_synonym['3'][index]),str(df_synonym['answer'][index]))
        predictions.append(prediction)
        
        labels.append(label)
    
    result_df['predictions'] = predictions
    result_df['label'] = labels
    pd.DataFrame(result_df).to_csv(f'{name}-details.csv', index=False)


def get_analysis(name, model):
    # Evaluates the model's performance 
    df_for_analysis = pd.read_csv(f'{name}-details.csv')

    label_counts = df_for_analysis['label'].value_counts()
    correct = label_counts.get('correct', 0)
    wrong = label_counts.get('wrong',0)
    guess = label_counts.get('guess', 0)
    v = correct + wrong
    line = f'{name}, {len(model.index_to_key)}, {correct}, {v}, {correct/v}'
    return line

def write_analysis(line):
    # Writes the model's performance to a csv file
    with open('analysis.csv', 'a+') as my_file:
        my_file.write("Model name, vocabulary size, # of correct labels (C), # of questions answered without guessing (V), model accuracy (C/V) \n"
                      + line + '\n')
        
    
    



In [10]:
# Task 1
def analysis_part_one():
    write_model_to_csv('word2vec-google-news-300',wv)
    line = get_analysis('word2vec-google-news-300',wv)
    write_analysis(line)
analysis_part_one()

KeyError: peacefulness
KeyError: harshness
KeyError: weariness
KeyError: happiness
